# Aldi vs. Lidl scraping

### Objectives

 - get list of postcodes
 - drop duplicates
 - split postcodes where 6 or 7 characters
 - drop rows where lat,long == 99,0

In [1]:
import pandas as pd
import requests
import time
import json

In [2]:
# Remove unnecessary data and save as new csv
my_list = pd.read_csv('.ONSPD_NOV_2021_UK/Data/ONSPD_NOV_2021_UK.csv',skipinitialspace=True)

In [3]:
# create new dataframe from loaded csv
x = my_list

# format dataframe and save to csv
x = x[['pcd', 'lat', 'long']].reset_index(drop=True)
x[['pc1','pc2']] = x['pcd'].str.split(expand=True)
x.to_csv('postcodes.csv')

# x['pc3'] = x['pcd'].str.replace(' ','')

In [4]:
# drop duplicates from dataframe
y = x.drop_duplicates(subset='pc1')
y['pc1'] = y['pcd'].str[:4]
y['pc2'] = y['pcd'].str[3:8]
y = y[y.lat < 99]
y = y.drop_duplicates(subset='pc1').reset_index(drop=True)
y['pc1'] = y['pc1'].str.replace(" ","")
y['pc2'] = y['pc2'].str.replace(" ","")
y.to_csv('postcodes_cut.csv')

# z = y['pc3'].squeeze().str.len()
# print(z)
# if length of string in pc3 == 7
# pc2 = pc1.split(4 characters in)
# print(y.iloc[:,0])

### Data download
 - Loop through postcode/lat/long to download Aldi store data
 - Drop duplicates

In [5]:
# Test payload
url = 'https://www.aldi.co.uk/api/store-finder/search?latitude=53.6064826&longitude=-2.4095537'
payload = {}
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
result = requests.get(url, headers=headers).json()

df = pd.json_normalize(result, record_path='results')
df = df.drop(['distance','description','openingTimes','isFirstStore','selectedPage'],axis=1).drop_duplicates('code')
df[['retailer', 'location']] = df['name'].str.split('-', 1, expand=True)
df.drop(['name', 'retailer'],axis=1, inplace=True)
df

,code,address,features,latlng.lat,latlng.lng,location
0,s-uk-E0125,"[Bolton, BL1 2HJ]",[parking],53.585353,-2.430781,Higher Bridge Street
1,s-uk-E1060,"[Bolton, BL2 6PP]",[parking],53.580825,-2.381599,"Fold Lane, Breightmet"
2,s-uk-E0623,"[Bolton, BL3 6DH]",[parking],53.573461,-2.433047,Trinity Street
3,s-uk-E1183,"[Bolton, BL3 2NB]",[],53.565766,-2.414645,Vincent Way
4,s-uk-E0879,"[Bury, BL8 1GU]",[parking],53.595568,-2.311532,"Crostons R Park, Wood St"


In [6]:
mythings = []
for row in y.itertuples():
    url = f'https://www.aldi.co.uk/api/store-finder/search?latitude={getattr(row, "lat")}&longitude={getattr(row, "long")}'
    mythings.append(url)

pd.DataFrame(data=mythings)

,0
0,https://www.aldi.co.uk/api/store-finder/search...
1,https://www.aldi.co.uk/api/store-finder/search...
2,https://www.aldi.co.uk/api/store-finder/search...
3,https://www.aldi.co.uk/api/store-finder/search...
4,https://www.aldi.co.uk/api/store-finder/search...
...,...
3047,https://www.aldi.co.uk/api/store-finder/search...
3048,https://www.aldi.co.uk/api/store-finder/search...
3049,https://www.aldi.co.uk/api/store-finder/search...
3050,https://www.aldi.co.uk/api/store-finder/search...


In [21]:
dump = []
for h,g in enumerate(mythings):
    result = requests.get(g, headers=headers)
    dump.append(result.json())

In [48]:
test = pd.DataFrame(data=dump).dropna().reset_index()
s1 = json.dumps(test['results'].to_dict())

# test2 = pd.DataFrame(s1.results.apply(json.load))
# df_final = pd.json_normalize(test.attributes.apply(json.loads), record_path='results')

In [26]:
test.columns

Index(['query', 'center', 'bounds', 'formattedCount', 'results', 'pagination'], dtype='object')

In [114]:
# new_test = pd.DataFrame(dump)
# new_test = new_test.dropna(how='all',subset='results')

KeyError: ['results']

In [ ]:
# df = pd.json_normalize(dump, record_path='results', errors='ignore')
# df = df.drop(['distance','description','openingTimes','isFirstStore','selectedPage'],axis=1).drop_duplicates('code')
# df[['retailer', 'location']] = df['name'].str.split('-', 1, expand=True)
# df[['town','postcode']] = pd.DataFrame(df.address.tolist(),index=df.index)

# df.drop(['name', 'retailer', 'address'],axis=1,inplace=True)
# df.to_json('test.json')
# df = df.reset_index(drop=True)
# df

###############

# for i in range(len(my_list)):
#   print(my_list.iloc[i, 3], my_list.iloc[i, 4])

In [105]:
new_test.reset_index(drop=True)
new_test

,query,center,bounds,formattedCount,results,pagination
0,,"{'lat': '57.101474', 'lng': '-2.242851'}","{'n': '57.28805', 'e': '-2.08268', 's': '56.91...",found near your current location,"[{'code': 's-uk-E0553', 'name': 'ALDI - Westhi...","{'next': 1, 'pages': [{'page': 0, 'prettyPage'..."
1,,"{'lat': '57.148232', 'lng': '-2.096648'}","{'n': '57.28805', 'e': '-1.813279', 's': '57.0...",found near your current location,"[{'code': 's-uk-E0673', 'name': 'ALDI - 9-10 B...","{'next': 1, 'pages': [{'page': 0, 'prettyPage'..."
2,,"{'lat': '57.145897', 'lng': '-2.095221'}","{'n': '57.28805', 'e': '-1.810425', 's': '57.0...",found near your current location,"[{'code': 's-uk-E0673', 'name': 'ALDI - 9-10 B...","{'next': 1, 'pages': [{'page': 0, 'prettyPage'..."
3,,"{'lat': '57.12785', 'lng': '-2.095537'}","{'n': '57.28805', 'e': '-1.8110570000000004', ...",found near your current location,"[{'code': 's-uk-E0673', 'name': 'ALDI - 9-10 B...","{'next': 1, 'pages': [{'page': 0, 'prettyPage'..."
4,,"{'lat': '57.098044', 'lng': '-2.239155'}","{'n': '57.28805', 'e': '-2.08268', 's': '56.90...",found near your current location,"[{'code': 's-uk-E0553', 'name': 'ALDI - Westhi...","{'next': 1, 'pages': [{'page': 0, 'prettyPage'..."
...,...,...,...,...,...,...
392,,"{'lat': '54.58057', 'lng': '-5.935161'}","{'n': '55.63185', 'e': '-4.48293', 's': '53.52...",found near your current location,"[{'code': 's-uk-E0627', 'name': 'ALDI - Goods ...","{'next': 1, 'pages': [{'page': 0, 'prettyPage'..."
396,,"{'lat': '54.602717', 'lng': '-5.922494'}","{'n': '55.63185', 'e': '-4.48293', 's': '53.57...",found near your current location,"[{'code': 's-uk-E0627', 'name': 'ALDI - Goods ...","{'next': 1, 'pages': [{'page': 0, 'prettyPage'..."
397,,"{'lat': '54.890559', 'lng': '-2.943791'}","{'n': '55.11607800000001', 'e': '-2.628428', '...",found near your current location,"[{'code': 's-uk-E0885', 'name': 'ALDI - Dalsto...","{'next': 1, 'pages': [{'page': 0, 'prettyPage'..."
398,,"{'lat': '54.7487', 'lng': '-2.70615'}","{'n': '54.91503', 'e': '-2.0371', 's': '54.582...",found near your current location,"[{'code': 's-uk-E0246', 'name': 'ALDI - Cavend...","{'next': 1, 'pages': [{'page': 0, 'prettyPage'..."


In [50]:
new_df = pd.read_json('test.json')
new_df

,code,features,latlng.lat,latlng.lng,location,town,postcode
0,s-uk-E0553,[parking],57.154870,-2.278550,Westhill Drive,Westhill,AB32 6FY
1,s-uk-E1423,[],57.064895,-2.136468,Portlethen Retail Park,Portlethen,AB124XP
2,s-uk-E1088,[],57.159397,-2.142220,10 Cornhill Shopping Arcade,Aberdeen,AB16 5UT
3,s-uk-E0673,[parking],57.149280,-2.082680,9-10 Boulevard Retail Park,Aberdeen,AB11 5EJ
4,s-uk-E0618,[],57.288050,-2.380017,Loco Works Road,Inverurie,AB51 4FY
46,s-uk-E0843,[parking],57.363450,-2.075120,Ythan Terrace,Ellon,AB41 9BD
